## Rolling Horizon Scheduler Model

##### Sets
Define the sets used in the model:
- $I$: Range of surgeries.
- $J$: Range of days.
- $K$: Range of rooms.

#### Variables
Define the decision variables:
- $X_{ijk}$: Binary variable indicating if surgery $i$ is scheduled on day $j$ in room $k$.
- $SWT_{i}$: Scheduled wait time for surgery $i$.
  - $SWT_{i} = j$ (where $X_{ijk} = 1$) - $AD_i$
- $Y_i$: Binary variable indicating whether the scheduled wait time for surgery $i$ exceeds the target wait time.

#### Parameters
Define the constants used in the model:
- $C_{jk}$: Room capacity on day $j$.
- $D_i$: Surgery $i$ duration.
- $T_i$: Target wait time for surgery $i$.
- $AD_i$: Admitted date for surgery $i$.
- $M$: A sufficiently large constant.

#### Constraints
Define the constraints:
- **Constraint 1: Surgery can only be done once**
  - $\sum_{jk} (X_{ijk}) = 1$ for all $i \in I$
- **Constraint 2: Room capacity**
  - $C_{jk} \geq \sum_{i \in I} (X_{ijk} \cdot D_i)$ for all $j$ in $J$, $k$ in $K$
- **Big M Constraint**: $SWT_{i} - T_{i} \leq M \cdot Y_{ijk}$ for all $i \in I$, $j$ in $J$, $k$ in $K$

#### Objective Function
Define the objective function:
- **Minimize** $\text{Minimize} \sum_{i,j,k} \left( \text{max}\left(0, X_{ijk} \cdot (SWT_i - T_i)\right) + \beta \cdot SWT_i \right)$



In [3]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import sys as sys

# Create a new model
model = gp.Model("Rolling_Horizon_Scheduler")

#Read In Surgical Information
surgicalPortfolio = pd.read_csv('surgeries_data.csv')
surgicalPortfolio['ID'] = pd.Series(surgicalPortfolio['id'] +" " + surgicalPortfolio['type'])
surgeries = surgicalPortfolio['ID']

#read in capacity information
caps = pd.read_csv('OR_caps.csv')
rooms = caps['room'].unique().tolist()
days = caps['day'].unique().tolist()

# Sets
I = surgeries
J = days
K = rooms 

# Parameters

C = {(row['day'], row['room']): row['capacity'] for _, row in caps.iterrows()} #capacities for each room on each day
D = {row['ID']: row['duration'] for _, row in surgicalPortfolio.iterrows()} # 
T = {row['ID']: row['ideal_waiting_time'] for _, row in surgicalPortfolio.iterrows()}
AD = {row['ID']: row['days_waited'] for _, row in surgicalPortfolio.iterrows()}
M = 10**99

# Variables
X = {(i, j, k): model.addVar(vtype=GRB.BINARY, name=f"X_{i}_{j}_{k}") for i in I for j in J for k in K}

SWT = {i: model.addVar(vtype=GRB.CONTINUOUS, name=f"SWT_{i}") for i in I}

Y = {i: model.addVar(vtype=GRB.BINARY, name=f"Y_{i}") for i in I}

# Constraints
# Constraint 1: Surgery can only be done once
for i in I:
    model.addConstr(gp.quicksum(X[i, j, k] for j in J for k in K) <= 1, name=f"One Surgery{i}")

# Constraint 2: Room capacity
for j in J:
    for k in K:
        model.addConstr(gp.quicksum(X[i, j, k] * D[i] for i in I) <= C[j, k], name=f"Capacity{j}_{k}")

#Constraints for variables: 
# Add constraint to set SWT[i] equal to the scheduled wait time plus the admitted date
for i in I:
    model.addConstr(SWT[i] == gp.quicksum(j * X[i, j, k] for j in J for k in K) * (1 + AD[i]),
                    name=f"SWT_Constraint_{i}")
    
# Define a new binary decision variable to indicate whether SWT[i] > T[i]
for i in I:
    model.addConstr(SWT[i] - T[i] <= M * Y[i], name=f"Constraint_SWT_T_{i}")

# Add constraints to enforce SWT[i] > T[i]
for i in I:
    model.addConstr(SWT[i] - T[i] <= M * Y[i])

# Define the penalty term for surgeries not scheduled at least once
penalty_unscheduled = gp.quicksum((1 - gp.quicksum(X[i, j, k] for j in J for k in K)) for i in I)

# Define the penalty term for surgeries exceeding target wait time
penalty_term = gp.quicksum((SWT[i] - T[i]) * Y[i] for i in I for j in J for k in K)

# Define the cost term for each scheduled surgery
cost_term = gp.quicksum(SWT[i] for i in I)

# Combine the penalty term and cost term to form the objective function
beta = 0.1
delta = 99999
model.setObjective(penalty_term + (beta * cost_term) + delta * penalty_unscheduled, GRB.MINIMIZE)


In [4]:
# Optimize the model
model.optimize()

# Check if the optimization was successful
if model.status == GRB.OPTIMAL:
    # Extract and print the schedule results
    print("Schedule Results:")
    for i in I:
        for j in J:
            for k in K:
                # Check if surgery i is scheduled on day j in room k
                if X[i, j, k].x > 0.5:  # Assuming x > 0.5 indicates scheduling
                    print(f"Surgery {i} is scheduled on day {j} in room {k}")
    
    # Extract and print the scheduled wait times
    print("\nScheduled Wait Times:")
    for i in I:
        print(f"Scheduled wait time for Surgery {i}: {SWT[i].x}")
else:
    print("No feasible solution found.")
    # Print all constraints
    for constr in model.getConstrs():
        print(constr)


Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: 13th Gen Intel(R) Core(TM) i7-1355U, instruction set [SSE2|AVX|AVX2]
Thread count: 10 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 168 rows, 936 columns and 2772 nonzeros
Model fingerprint: 0xe439ccfb
Model has 36 quadratic objective terms
Variable types: 36 continuous, 900 integer (900 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+99]
  Objective range  [1e-01, 1e+05]
  QObjective range [5e+01, 5e+01]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+00]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Found heuristic solution: objective 3599964.0000
Found heuristic solution: objective 3598884.0000
Presolve removed 36 rows and 0 columns
Presolve time: 0.00s
Presolved: 168 rows, 972 columns, 2808 nonzeros
Variable types: 0 continuous, 972 integer (900 binary)

Root r